## 上交所代码

In [72]:
import requests
import csv
import re

In [3]:
## turn to next page fuction 
def downlourl(currentpage):
    url = "http://query.sse.com.cn/commonSoaQuery.do?siteId=28&sqlId=BS_GGLL&extGGLX=&stockcode=&channelId=10743%2C10744%2C10012&extGGDL=&order=createTime%7Cdesc%2Cstockcode%7Casc&isPagination=true&pageHelp.pageSize=15&pageHelp.pageNo=" + repr(currentpage) + "&pageHelp.beginPage=" + repr(currentpage) +"&pageHelp.cacheSize=1"
    return(url)

In [ ]:
#asynchronous loadnig 
headers = {
    'Referer':'http://www.sse.com.cn/disclosure/credibility/supervision/inquiries/',
    'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'
}


In [ ]:
#generating target csv 
with open('sh.csv',"w",newline='') as f:
    writer = csv.writer(f, delimiter=',')
    title=['时间2','标题','公司代码','函件类别','公司简称','函件类型','时间1','网址','函件编码']
    writer.writerow(title)
    for page in range(1,148):
        r = requests.get(downlourl(page), headers=headers)
        for i in r.json()['result']:
            result=re.search('c/(\d+).pdf',i['docURL'])
            writer.writerow([i['cmsOpDate'],i['docTitle'],i['stockcode'],i['extWTFL'],i['extGSJC'],i['docType'],i['createTime'],i['docURL'],re.search('c/(\d+).pdf',i['docURL']).group(1)])
     

In [ ]:
#生成csv (只有5个required columns)
with open('上交所.csv',"w",newline='') as f:
    writer = csv.writer(f, delimiter=',')
    title=['公司代码','公司简称','发函日期','监管问询类型','标题']
    writer.writerow(title)
    for page in range(1,148):
        r = requests.get(downlourl(page), headers=headers)
        for i in r.json()['result']:
            result=re.search('c/(\d+).pdf',i['docURL'])
            writer.writerow([i['stockcode'],i['extGSJC'],i['createTime'],i['extWTFL'],i['docTitle']])
    

In [ ]:
import pandas as pd
import time
from urllib.request import urlopen
from urllib.request import Request
from urllib.request import quote
import requests
import pdfplumber
import re

In [ ]:
#download and inspect each file 
data = pd.read_csv("sh2.csv",encoding='utf-8')


函件编码 = data.loc[:,'函件编码']
网址 = data.loc[:,'网址']
函件类型 = data.loc[:,'函件类型']

headers = {'content-type': 'application/json',
           'Accept-Encoding': 'gzip, deflate',
           'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:53.0) Gecko/20100101 Firefox/53.0'}

baseurl = "http://reportdocs.static.szse.cn/UpFiles/fxklwxhj/"

In [ ]:
##parse in the documenbt 
def parse(docucode):

    _path = "http://" + quote(docucode)
    print(_path)

    resource = requests.get(_path, stream=True)
    with open('/Users/bingcuiguo/Desktop/oa/'+re.search('c/(\d+.pdf)', docucode).group(1),'wb') as fd:
        for y in resource.iter_content(102400):
            fd.write(y)
        print(re.search('c/(\d+.pdf)', docucode).group(1), 'Complete Download')
    path = '/Users/bingcuiguo/Desktop/oa/'+ re.search('c/(\d+.pdf)', docucode).group(1)
    pdf = pdfplumber.open(path)
    f = open("/Users/bingcuiguo/Desktop/oa/" + re.search('c/(\d+).pdf', docucode).group(1) + '.txt', 'w')
    f2 = open("/Users/bingcuiguo/Desktop/oa/" +  'anomaly_record.txt', 'a')
    n=1
    for page in pdf.pages:
        n+=1
        print(page.extract_text())
        try:
            f.writelines(page.extract_text())
        except:
            print(f'========={docucode}第{n}页解码异常==========')
            f2.write(f'\n ========={docucode}第{n}页解码异常==========')

In [ ]:
##Download pdf and save each file to txt 
for i in range(len(网址)):
    函件名称 =re.search('c/(\d+.pdf)', 网址[i]).group(1)
    print(函件名称)
    开始爬取时间 = "这是第%d个公告"%i
    print(开始爬取时间)
    print(time.strftime('%Y.%m.%d.%H:%M:%S',time.localtime(time.time())))
    try:
        if 函件类型[i]=="pdf":
            parse(网址[i])
            print(函件名称 + "Success")
        else:
            with open("/Users/bingcuiguo/Desktop/oa"%函件名称,'wb') as f:
                _path = baseurl + quote(函件名称) +"?random=0.3006649122149502"
                request = requests.get(url=_path, headers=headers)  # 随机从user_agent列表中抽取一个元素
                f.write(request.content)
        结束爬取时间 = time.strftime('%Y.%m.%d.%H:%M:%S', time.localtime(time.time()))
        print(结束爬取时间)
        print("第%d个公告爬取完成" % i)
    except:
        time.sleep(5)
        try:
            if 函件类型[i]=="pdf":
                parse(网址[i])
                print(函件名称 + "Success")
            else:
                with open("/Users/bingcuiguo/Desktop/oa/%s"%函件名称,'wb') as f:
                    _path = baseurl + quote(函件名称) +"?random=0.3006649122149502"
                    request = requests.get(url=_path, headers=headers)  # 随机从user_agent列表中抽取一个元素
                    f.write(request.content)
            结束爬取时间 = time.strftime('%Y.%m.%d.%H:%M:%S', time.localtime(time.time()))
            print(结束爬取时间)
            print("第%d个公告爬取完成" % i)
        except:
            f2 = open("/Users/bingcuiguo/Desktop/oa" + 'anomaly_record.txt', 'a')
            f2.write(f'\n ========={函件名称}爬取时网络异常==========')

In [71]:
## Put txt into a single excel file 
import os
import docx2txt
from openpyxl import Workbook

In [73]:
content_list = []

wb = Workbook()
sheet = wb.active
sheet['A1'].value = '公告编码'
sheet['A2'].value = '公告内容'

In [74]:
def readdocx(filepath):
    content = docx2txt.process(filepath)  #打开传进来的路径
    docucode = filepath.split('/')[-1]
    content_list.append([docucode.split('.')[0],content])
    content_list.append([docucode.split('.')[0],content])

In [75]:
def readtxt(filepath):
    content = open(filepath, "r").read()
    docucode = filepath.split('/')[-1]
    content_list.append([docucode.split('.')[0],content])


In [76]:
def eachFile(filepath):
    pathDir = os.listdir(filepath) ## return current path 
    for s in pathDir:
        newDir=os.path.join(filepath,s)#add filename to path 
        if os.path.isfile(newDir) :     
            doctype = os.path.splitext(newDir)[1]
            if doctype == ".txt":  
                readtxt(newDir)
            elif doctype == ".docx":
                readdocx(newDir)
            else:
                pass
        else:
            eachFile(newDir) 

In [ ]:
filepath="/Users/bingcuiguo/Desktop/oa/上交所"
eachFile("/Users/bingcuiguo/Desktop/oa/上交所")
a = 1
for doc in content_list:
    sheet['A%d'%a].value = doc[0]
    sheet['B%d'%a].value = doc[1]
    a += 1
wb.save('上交所.xlsx')

In [77]:
##create database
import sqlite3

In [ ]:
import pandas as pd
from pandas import DataFrame
data = pd.read_csv("sh.csv")
data = pd.DataFrame(data, columns = ['时间2','标题','公司代码','函件类别','公司简称','函件类型','时间1','网址','函件编码'])

In [ ]:
conn = sqlite3.connect('TestDB.db')
c = conn.cursor()

In [ ]:
c.execute('''
CREATE TABLE 上交所 (
时间2,标题,公司代码,函件类别,公司简称,函件类型,时间1,网址,函件编码)
''')

In [ ]:
with open('sh.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row in csv_reader:
        time2 = row[0]
        title = row[1]
        company_code = row[2]
        file_type = row[3]
        company_abbr = row[4]
        file_type2 = row[5]
        time1 = row[6]
        web = row[7]
        file_code = row[8]
        c.execute('''INSERT INTO 上交所 (时间2,标题,公司代码,函件类别,公司简称,函件类型,时间1,网址,函件编码)
        VALUES (?,?,?,?,?,?,?,?,?)
        ''',(time2, title, company_code, file_type, company_abbr, file_type2, time1, web, file_code))
    conn.commit()

In [ ]:
c.execute('''
select 公司代码,公司简称,时间1 as 发函日期, 函件类型,标题 from 上交所;
''')

In [ ]:
print(c.fetchall())

In [ ]:
content = pd.read_excel('上交所.xlsx')

In [ ]:
c.execute('''
CREATE TABLE 公报内容 (
函件编码，内容)
''')

In [ ]:
content.to_csv('内容.csv', encoding='utf-8')

In [ ]:
with open('内容.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row in csv_reader:
        content_code = row[0]
        content = row[1]
        c.execute('''INSERT INTO 公报内容 (函件编码,内容)
        VALUES (?,?)
        ''',(content_code, content))
    conn.commit()

In [ ]:
c.execute('''
select 公司代码,公司简称,时间1 as 发函日期, 函件类型,标题 from 
上交所 a left join (select 内容 from 公报内容) b on a.函件编码 = b.函件编码
''')
print(c.fetchall())

## 深交所代码

In [35]:
##通过根据切换tab_num来切换板块
def downlourl(currentpage, tab_num):
    url="http://www.szse.cn/api/report/ShowReport/data?SHOWTYPE=JSON&CATALOGID=main_wxhj&TABKEY=tab"+ str(tab_num)+"&PAGENO="+ str(currentpage)+"&random=0.7562589043142469"
    return(url)
headers = {
    'Referer':'http://www.szse.cn/disclosure/supervision/inquire/index.html',
    'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.146 Safari/537.36'
}


In [ ]:
page_range = [106, 146, 144]
with open('sz.csv',"w",newline='') as f:
    title=['公司代码','公司简称','发函日期','函件类别','函件编码','公司回复','函件类型']
    writer = csv.writer(f, delimiter=',')
    writer.writerow(title)
    for tab in range(1,4):
        for page in range(1,page_range[tab-1]):
            r = requests.get(downlourl(page, tab), headers=headers)
            for i in r.json()[0]['data']:
                ck=re.search("encode-open=(.*?)>",i['ck'])
                hfck=re.search(">(.*?)<",i['hfck'])
                lx=(ck.group(1)).split(".")[1][:-1]
                if hfck:
                    # print(hfck.group(1))
                    hfck=hfck.group(1)
                try:
                    writer.writerow([i['gsdm'],i['gsjc'],i['fhrq'],i['hjlb'],(ck.group(1))[19:-5],hfck,lx])
                except:
                    f2 = open("/Users/bingcuiguo/Desktop/oa" + 'anomaly_record.txt', 'a')
                    f2.write(f'\n ========={(ck.group(1))[19:-5]}解码异常==========')
                # writer.writerow([(ck.group(1))[19:-5],lx])
            print('完成爬取第%d页'%page+ '第%d个tab'%tab)

In [8]:
import pandas as pd
import time
from urllib.request import urlopen
from urllib.request import Request
from urllib.request import quote
import requests
from pdfminer.converter import PDFPageAggregator
from pdfminer.layout import LTTextBoxHorizontal, LAParams
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfpage import PDFTextExtractionNotAllowed

In [9]:
data = pd.read_csv("sz.csv",encoding='utf-8')


函件编码 = data.loc[:,'函件编码']
函件类型 = data.loc[:,'函件类型']

headers = {'content-type': 'application/json',
           'Accept-Encoding': 'gzip, deflate',
           'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:53.0) Gecko/20100101 Firefox/53.0'}


<method 'read' of '_io.BytesIO' objects>

In [16]:
baseurl = "http://reportdocs.static.szse.cn/UpFiles/fxklwxhj/"
from io import BytesIO

def parse(docucode):
    # open pdf online
    _path = baseurl + quote(docucode) +"?random=0.3006649122149502"
    request = Request(url=_path, headers=headers)  # 随机从user_agent列表中抽取一个元素
    fp = urlopen(request)
    fp = BytesIO(fp.read())
    # create a parser for pdf 
    parser_pdf = PDFParser(fp)
    # create a phdf
    doc = PDFDocument(parser_pdf)
    parser_pdf.set_document(doc)
    doc.set_parser(parser_pdf)
    ##doc initialization
    doc.initialize()
    # detect whether it could be converted to txt 
    if not doc.is_extractable:
        raise PDFTextExtractionNotAllowed
    else:
        rsrcmgr = PDFResourceManager()
        laparams = LAParams()
        device = PDFPageAggregator(rsrcmgr, laparams=laparams)
        interpreter = PDFPageInterpreter(rsrcmgr, device)
        #for each page, get the page content and column 
        for page in doc.get_pages():
            # use interpreter to analyze 
            interpreter.process_page(page)
            # use aggregator to get content 
            layout = device.get_result()
            for out in layout:
                docname = "/Users/bingcuiguo/Desktop/oa/深交所/"+str(docucode).split('.')[0]+'.txt'
                with open(docname,'a') as f:
                    if isinstance(out, LTTextBoxHorizontal):
                        results = out.get_text()
                        print(results)
                        try:
                            f.write(results)
                        except:
                            
                            print(f'========={docucode}decode anomaly==========')
                            f3 = open("/Users/bingcuiguo/Desktop/oa" + 'anomaly_record.txt', 'r')
                            f3.write(f'\n ========={docucode}页 decode anomaly==========')


In [17]:
for i in range(len(函件编码)):
    函件名称 = (函件编码[i] + '.' + 函件类型[i])
    print(函件名称)
    开始爬取时间 = "这是第%d个公告"%i
    print(开始爬取时间)
    print(time.strftime('%Y.%m.%d.%H:%M:%S',time.localtime(time.time())))
    if 函件类型[i]=="pdf":
        parse(函件名称)
        print(函件名称 + "爬取成功")
    else:
        with open("/Users/bingcuiguo/Desktop/oa/深交所/%s"%函件名称,'wb') as f:
            _path = baseurl + quote(函件名称) +"?random=0.3006649122149502"
            request = requests.get(url=_path, headers=headers)  # 随机从user_agent列表中抽取一个元素
            f.write(request.content)
    结束爬取时间 = time.strftime('%Y.%m.%d.%H:%M:%S', time.localtime(time.time()))
    print(结束爬取时间)
    print("第%d个公告爬取完成" % i)

CDD00080659352.pdf
这是第0个公告
2021.02.04.23:44:08


AttributeError: 'PDFDocument' object has no attribute 'set_parser'

In [ ]:
content_list = []

wb = Workbook()
sheet = wb.active
sheet['A1'].value = '公告编码'
sheet['A2'].value = '公告内容'

In [ ]:
filepath="/Users/bingcuiguo/Desktop/oa/深交所"
eachFile("/Users/bingcuiguo/Desktop/oa/深交所")
a = 1
for doc in content_list:
    sheet['A%d'%a].value = doc[0]
    sheet['B%d'%a].value = doc[1]
    a += 1
wb.save('深交所.xlsx')

In [ ]:
import pandas as pd
from pandas import DataFrame
data = pd.read_csv("sz.csv")
data = pd.DataFrame(data, columns = ['公司代码','公司简称','发函日期','函件类别','函件编码','公司回复','函件类型'])

In [ ]:
conn = sqlite3.connect('TestDB.db')
c = conn.cursor()

In [ ]:
c.execute('''
CREATE TABLE 深交所 (
公司代码,公司简称,发函日期,函件类别,函件编码,公司回复,函件类型)
''')

In [ ]:
with open('sz.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row in csv_reader:
        company_code = row[0]
        company_abbr = row[1]
        time = row[2]
        file_type = row[3]
        file_code = row[4]
        company_feedback = row[5]
        file_type2 = row[6]
        c.execute('''INSERT INTO 深交所 (公司代码,公司简称,发函日期,函件类别,函件编码,公司回复,函件类型)
        VALUES (?,?,?,?,?,?,?)
        ''',(compnay_code, company_abbr, time, file_type, file_code, company_feedback, file_type2))
    conn.commit()

In [ ]:
content = pd.read_excel('深交所.xlsx')

In [ ]:
c.execute('''
CREATE TABLE 公报内容 (
函件编码，内容)
''')

In [ ]:
content.to_csv('深交所内容.csv', encoding='utf-8')

In [ ]:
with open('深交所内容.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row in csv_reader:
        content_code = row[0]
        content = row[1]
        c.execute('''INSERT INTO 公报内容 (函件编码,内容)
        VALUES (?,?)
        ''',(content_code, content))
    conn.commit()

In [ ]:
c.execute('''
select 公司代码,公司简称,时间1 as 发函日期, 函件类型 from 
深交所 a left join (select 内容 from 公报内容) b on a.函件编码 = b.函件编码
''')
print(c.fetchall())

#### 其他可用方法：使用selenium模拟下载
#### 可以提升的地方：使用multiprocessing多线程完成下载和解析可以更快的提高速度